In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [5]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv') #讀資料夾底下的csv
    symbol=etf_csv['Symbol']#all etf
    inception=etf_csv['Inception'] # all etf inception time
    count=0
    real_symbol=[] #etf
    in_s=[] #time
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

In [6]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+ETF+"?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk' or freq == '1mo'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [7]:
etfs=get_etf()
etfs.remove('ZJPN')
etfs.remove('PAF')# 移除不合規定的etf
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target,etfs):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    for Symbol in etfs:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data

In [8]:
all_data_week = get_Date('1wk','adjclose',etfs)
all_data_month = get_Date('1mo','adjclose',etfs)

----------1wk----------
start crawling : EWJ
start crawling : EWY
start crawling : FNDF
start crawling : VPL
start crawling : DXJ
start crawling : EWT
start crawling : EWH
start crawling : EPP
start crawling : FNDC
start crawling : DLS
start crawling : JPIN
start crawling : PXF
start crawling : GSIE
start crawling : EWA
start crawling : FDT
start crawling : IPAC
start crawling : GWX
start crawling : DWM
start crawling : DFJ
start crawling : HEWJ
start crawling : DBJP
start crawling : EWS
start crawling : DOL
start crawling : HFXI
start crawling : PDN
start crawling : IVLU
start crawling : IDOG
start crawling : DTH
start crawling : DIM
start crawling : DOO
start crawling : SCJ
start crawling : ENZL
start crawling : IDLB
start crawling : FJP
start crawling : JPXN
start crawling : IVAL
start crawling : DXJS
start crawling : ISCF
start crawling : DEEF
start crawling : IMOM
start crawling : JPN
start crawling : IDHQ
start crawling : FPA
start crawling : DVYA
start crawling : JPMV
start craw

In [9]:
all_data_week=all_data_week.dropna()
#all_data_week

In [10]:
#all_data_month

In [11]:
all_data_month=all_data_month.dropna()
#all_data_month

In [12]:
def  calculate_ra_1(etf): # calculate one ra
    today_etf=etf[-1]
    return [(today_etf-etf[i])/etf[i] for i in range(0,len(etf)-1)]

In [13]:
def calculate_ra_all(date,etfs,data):
    dict={}
    for etf in etfs:
        dict[etf]=calculate_ra_1(data[etf])
    
    df = pd.DataFrame.from_dict(dict)
    return df

In [14]:
returns=calculate_ra_all(all_data_month.index,etfs,all_data_month)

In [15]:
week_returns=calculate_ra_all(all_data_week.index,etfs,all_data_week)
#week_returns

In [16]:
# returns=calculate_ra_all(all_data_month.index,etfs)
# returns

In [17]:
def OmegaSharpeRatio(R, L):
    L=L/100
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [18]:
def ranking(etfs,returns,date):
    arr=[]
    symbols=[]
    if date=='week':
        rf=2.24/4
    else:
        rf=2.24
    for etf in etfs:
        if OmegaSharpeRatio(returns[etf],rf)!="undefined":
            arr.append(OmegaSharpeRatio(returns[etf],rf))
            symbols.append(etf)
    dict_omega={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_omega)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df

In [19]:
data=ranking(etfs,week_returns,'week')
data

,ETF_ Symbol,Value
Ranking,,
1,HAUD,2883.096131
2,EWH,817.184182
3,EPP,74.793238
4,SCIX,29.360763
5,EWA,25.142955
6,DOO,21.003610
7,FAUS,16.532639
8,IDHQ,13.957150
9,HFXI,13.154458


In [20]:
data=ranking(etfs,returns,'month')
data

,ETF_ Symbol,Value
Ranking,,
1,EWH,420.559737
2,HAUD,315.560366
3,EPP,44.908598
4,SCIX,26.698052
5,EWA,14.572445
6,DOO,5.288793
7,EWT,4.671522
8,FHK,4.642566
9,FAUS,4.328689


In [21]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASSR(etf, returns, b3):
    x = np.asarray(returns[etf])
    SR = mean(x)/np.std(x)
    S = skew(x)           #偏度
    ASSR = SR * cmath.sqrt(1+b3*((S/3)*SR))
    
    return ASSR

In [22]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASKSR(etf, returns, rf):
    x = np.asarray(returns[etf])
    u = mean(x)/x.size    #E
    v = var(x, ddof=1)    #Var
    SD = math.sqrt(v)     #std
    S = skew(x)           #skew
    K = kurtosis(x)       #k

    c1 = 3*K-4*pow(S,2)-9
    c2 = 3*K-5*pow(S,2)-9
    a = ( 3*(cmath.sqrt(c1) ) / ( v*c2 ) )
    b = ( 3*S ) / ( SD*c2 )
    n = u - ( (3*S*SD) / c1 )
    d = 3*SD * ( (cmath.sqrt(c2)) / (c1) ) 
    phi = cmath.sqrt(pow(a,2)*pow(b,2))
    a_ = b + ( a*(n-rf) / cmath.sqrt(pow(d,2)+pow(n-rf,2))) 
    ASKSR = cmath.sqrt(2*(a_*(n-rf)-d*(phi-cmath.sqrt(pow(a,2)-pow(b-a_,2)))))
    
    return ASKSR

In [23]:
def rankingByASKSR(etfs, returns):
    arr=[]
    symbols=[]
    for etf in etfs:
        if sharpeASKSR(etf, returns ,0)!="undefined":
            if sharpeASKSR(etf, returns ,0).imag == 0:
                arr.append(sharpeASKSR(etf, returns ,0))
                symbols.append(etf)
            else:
                arr.append(sharpeASSR(etf, returns ,0))
                symbols.append(etf)
    dict_ASKSR={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_ASKSR)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df
        

In [24]:
rankingByASKSR(etfs, returns)

NameError: name 'std' is not defined